In [2]:
import sys
sys.path.append("../src")
import matplotlib.pyplot as plt
import pickle
import torch
from torch_geometric.data import Data
from torch_geometric.utils import subgraph

%load_ext autoreload
%autoreload 2

from data import *
from loader import *
from visualize import plot_merger_tree
from typing import List
from collections import deque
import networkx as nx

from pathlib import Path
BASE_DIR = Path("../")
RESULTS_DIR = Path(BASE_DIR / "results")

# Prepare data

In [2]:
# from data.py
subhalos = prepare_subhalos()
len(subhalos)

/home/john/research/assembly-vs-environment/notebook/../src/data.py:94: RuntimeWarning: divide by zero encountered in log10
  np.column_stack([halo_id, subhalo_flag, np.arange(len(subhalo_stellarmass)), subhalo_pos, subhalo_vel, subhalo_n_stellar_particles, subhalo_gasmass, subhalo_stellarmass, subhalo_halomass, subhalo_stellarhalfmassradius, subhalo_vmax, np.log10(subhalo_MRhalf), np.log10(subhalo_MRvmax), np.log10(subhalo_MR2half), np.log10(subhalo_RMhalf), np.log10(subhalo_spin), np.log10(subhalo_Vdisp), np.log10(subhalo_VmaxRad), np.log10(subhalo_offset)]),
/home/john/miniforge3/envs/pyg/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/john/miniforge3/envs/pyg/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/john/miniforge3/envs/pyg/lib/python3

132953

In [3]:
subhalos.to_parquet(RESULTS_DIR / "subhalos.parquet")

## Env

In [8]:
# from loader.py
env_data = make_cosmic_graph(subhalos, 3) 
env_data

/home/john/research/assembly-vs-environment/notebook/../src/loader.py:121: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  index=torch.tensor(target_nodes),


Data(x=[132953, 11], edge_index=[2, 6653247], edge_attr=[6653247, 6], y=[132953, 2], pos=[132953, 3], vel=[132953, 3], is_central=[132953, 1], x_hydro=[132953, 2], pos_hydro=[132953, 3], vel_hydro=[132953, 3], halfmassradius=[132953, 1], subhalo_id=[132953], overdensity=[132953])

In [9]:
with open(RESULTS_DIR / "cosmic_graphs_3Mpc.pkl", "wb") as f:
    pickle.dump(env_data, f)

## Trees

In [11]:
# from data -- takes like 5 min (and 40+GB of memory)
all_trees = load_trees()


/home/john/research/assembly-vs-environment/notebook/../src/data.py:227: RuntimeWarning: divide by zero encountered in log10
  df["subhalo_MRhalf"] = np.log10(subhalo_MRhalf)
/home/john/research/assembly-vs-environment/notebook/../src/data.py:229: RuntimeWarning: divide by zero encountered in log10
  df["subhalo_MR2half"] = np.log10(subhalo_MR2half)
/home/john/research/assembly-vs-environment/notebook/../src/data.py:227: RuntimeWarning: divide by zero encountered in log10
  df["subhalo_MRhalf"] = np.log10(subhalo_MRhalf)
/home/john/research/assembly-vs-environment/notebook/../src/data.py:229: RuntimeWarning: divide by zero encountered in log10
  df["subhalo_MR2half"] = np.log10(subhalo_MR2half)
/home/john/research/assembly-vs-environment/notebook/../src/data.py:227: RuntimeWarning: divide by zero encountered in log10
  df["subhalo_MRhalf"] = np.log10(subhalo_MRhalf)
/home/john/research/assembly-vs-environment/notebook/../src/data.py:229: RuntimeWarning: divide by zero encountered in lo

In [ ]:
# from loader.py
tree_data = make_merger_tree_graphs(all_trees, subhalos)
len(tree_data)

N_not_in_subhalo_cat = 5396

N_is_satellite = 0

N_bad_stellar_mass = 5437

In [13]:
with open(RESULTS_DIR / "merger_trees.pkl", "wb") as f:
    pickle.dump(tree_data, f)

# Bonsai: Merger Tree Trimming

Trim the trees by removing secular evolution, i.e., remove nodes with exactly one incoming and one outgoing node, and then rewire the edges around them.

In [44]:

def trim_merger_trees(graphs: List[Data]) -> List[Data]:
    """
    Remove nodes with exactly one incoming and one outgoing edge from merger trees.
    """
    trimmed_graphs = []

    for graph in tqdm.tqdm(graphs):
        # find nodes to remove
        num_nodes = graph.num_nodes
        row, col = graph.edge_index

        out_degree = torch.zeros(num_nodes, dtype=torch.long, device=row.device)
        out_degree.scatter_add_(0, row, torch.ones_like(row))
        in_degree = torch.zeros(num_nodes, dtype=torch.long, device=col.device)
        in_degree.scatter_add_(0, col, torch.ones_like(col))

        nodes_to_remove = (in_degree == 1) & (out_degree == 1)

        # skip if no trimming required
        if not nodes_to_remove.any():
            trimmed_graphs.append(graph.clone())
            continue

        # use bfs to rewire the graph
        nodes_to_keep_mask = ~nodes_to_remove
        nodes_to_keep_indices = torch.where(nodes_to_keep_mask)[0]
        
        # create an adjacency list for efficient traversal
        adj = [[] for _ in range(num_nodes)]
        for i, j in graph.edge_index.t().tolist():
            adj[i].append(j)

        new_edges = []
        # find next reachable node
        for start_node in nodes_to_keep_indices.tolist():
            q = deque(adj[start_node])
            visited = set(adj[start_node])

            while q:
                current_node = q.popleft()
                # if this is a node to keep, we've found a new edge
                if nodes_to_keep_mask[current_node]:
                    new_edges.append((start_node, current_node))
                # otherwise, continue the search down this path
                else:
                    for neighbor in adj[current_node]:
                        if neighbor not in visited:
                            visited.add(neighbor)
                            q.append(neighbor)
        
        # handle this case if no edges left
        if not new_edges:
            new_edge_index = torch.empty((2, 0), dtype=torch.long, device=graph.edge_index.device)
        else:
            new_edge_index = torch.tensor(new_edges, device=graph.edge_index.device).t()
        
        # build a mapping tensor for fast, vectorized index conversion (old_idx -> new_idx)
        remap_tensor = torch.full((num_nodes,), -1, dtype=torch.long, device=row.device)
        remap_tensor[nodes_to_keep_indices] = torch.arange(len(nodes_to_keep_indices), device=row.device)
        
        final_edge_index = remap_tensor[new_edge_index]

        # create the new Data object
        trimmed_graph = Data(
            x=graph.x[nodes_to_keep_mask],
            edge_index=final_edge_index,
            root_subhalo_id=graph.root_subhalo_id,
            y=graph.y,
        )

        # recalculate edge attributes
        if 'edge_attr' in graph:
            trimmed_graph.edge_attr = (
                trimmed_graph.x[final_edge_index[1], -1] -
                trimmed_graph.x[final_edge_index[0], -1]
            ).view(-1, 1)

        trimmed_graphs.append(trimmed_graph)

    return trimmed_graphs

In [45]:
bonsais = trim_merger_trees(tree_data)

100%|█████████████████████████████████████████████████████| 123004/123004 [04:53<00:00, 418.73it/s]


In [48]:
with open(RESULTS_DIR / "merger_tree_bonsais.pkl", "wb") as f:
    pickle.dump(bonsais, f)

# Merger Tree Stumps

Only keep snapshots 90-99; maintain all secular evolution

In [7]:
def create_stumps(graphs: List[Data], min_snap: int = 90, max_snap: int = 99) -> List[Data]:
    """
    Hacks down merger trees to create "stumps" using the PyG subgraph utility.
    """
    stump_data = []
    for graph in tqdm.tqdm(graphs):
        # keep nodes
        snapshot_numbers = graph.x[:, -1]
        node_mask = (snapshot_numbers >= min_snap) & (snapshot_numbers <= max_snap)
        nodes_to_keep = node_mask.nonzero().squeeze(-1) # Get indices from mask

        if nodes_to_keep.numel() == 0:
            # empty graph if missing
            empty_stump = Data(
                x=torch.empty(0, graph.num_node_features),
                edge_index=torch.empty(2, 0, dtype=torch.long),
                y=graph.y, 
                root_subhalo_id=graph.root_subhalo_id,
            )
            stump_data.append(empty_stump)
            continue
            
        # make subgraph; reindex and relabel nodes
        stump_edge_index, stump_edge_attr = subgraph(
            subset=nodes_to_keep,
            edge_index=graph.edge_index,
            edge_attr=getattr(graph, 'edge_attr', None),
            relabel_nodes=True,
            num_nodes=graph.num_nodes,
        )

        # save new Data
        tree_stump = Data(
            x=graph.x[nodes_to_keep], # Use indices to select node features
            edge_index=stump_edge_index,
            y=graph.y,
            root_subhalo_id=graph.root_subhalo_id,
        )
        
        if stump_edge_attr is not None:
            tree_stump.edge_attr = stump_edge_attr

        stump_data.append(tree_stump)

    return stump_data

In [8]:
tree_stumps = create_stumps(tree_data)

100%|███████████████████████████████████████████████████| 123004/123004 [00:07<00:00, 15977.42it/s]


In [16]:
with open(RESULTS_DIR / "merger_tree_stumps.pkl", "wb") as f:
    pickle.dump(tree_stumps, f)

# Reload all

In [3]:
subhalos = pd.read_parquet(RESULTS_DIR / "subhalos.parquet")

with open(RESULTS_DIR / "cosmic_graphs_3Mpc.pkl", "rb") as f:
    env_data = pickle.load(f)

with open(RESULTS_DIR / "merger_trees.pkl", "rb") as f:
    tree_data = pickle.load(f)

# remove all secular evolution
with open(RESULTS_DIR / "merger_tree_bonsais.pkl", "rb") as f:
    bonsais = pickle.load(f)

# snapshots 90-99
with open(RESULTS_DIR / "merger_tree_stumps.pkl", "rb") as f:
    stumps = pickle.load(f)

In [59]:
subhalos.columns

Index(['halo_id_DMO', 'subhalo_flag_DMO', 'subhalo_id_DMO', 'subhalo_x_DMO',
       'subhalo_y_DMO', 'subhalo_z_DMO', 'subhalo_vx_DMO', 'subhalo_vy_DMO',
       'subhalo_vz_DMO', 'subhalo_n_stellar_particles_DMO',
       'subhalo_MRhalf_DMO', 'subhalo_MRvmax_DMO', 'subhalo_MR2half_DMO',
       'subhalo_RMhalf_DMO', 'subhalo_spin_DMO', 'subhalo_Vdisp_DMO',
       'subhalo_VmaxRad_DMO', 'subhalo_offset_DMO', 'is_central_DMO',
       'subhalo_logstellarmass_DMO', 'subhalo_loghalomass_DMO',
       'subhalo_logvmax_DMO', 'subhalo_logstellarhalfmassradius_DMO',
       'subhalo_loggasmass_DMO', 'halo_id', 'subhalo_id', 'subhalo_x',
       'subhalo_y', 'subhalo_z', 'subhalo_vx', 'subhalo_vy', 'subhalo_vz',
       'subhalo_n_stellar_particles', 'subhalo_MRhalf', 'subhalo_MRvmax',
       'subhalo_MR2half', 'subhalo_RMhalf', 'subhalo_spin', 'subhalo_Vdisp',
       'subhalo_VmaxRad', 'subhalo_offset', 'is_central',
       'subhalo_l_halo_tree', 'subhalo_sublink', 'subhalo_gasmetallicity',
       '

# Visualize data

## Double check stellar mass--halo mass relations

In [18]:
plt.figure(figsize=(4,4), dpi=150)

plt.scatter(env_data.x[:, 0], env_data.y[:, 0], s=1, edgecolors="none", label="Env SHMR")
plt.scatter(*list(zip(*[(tree.x[0][0].item(), tree.y[0].item()) for tree in tree_data])), s=1, edgecolors="none", label="Mergers SHMR")
plt.legend(markerscale=8)
plt.xlabel(r"log($M_{\rm subhalo}/M_{\odot}$)", fontsize=14)
plt.ylabel(r"log($M_{\bigstar}/M_{\odot}$)", fontsize=14)
plt.grid(alpha=0.15)

plt.text(0.95, 0.2, "Centrals", color="C3", fontsize=16, transform=plt.gca().transAxes, ha="right")
plt.text(0.95, 0.1, "Satellites", color="C0", fontsize=16, transform=plt.gca().transAxes, ha="right")

plt.clf()

<Figure size 600x600 with 0 Axes>

The point of the above: these are exactly on top of each other.

In [40]:
plt.figure(figsize=(4,4), dpi=300)

plt.scatter(
    env_data.x[:, 0].numpy(), 
    env_data.y[:, 0].numpy(), 
    c=np.where(env_data.is_central.numpy().flatten(), "C3", "C0"), 
    s=1,
    edgecolors="none",
    rasterized=True
)
plt.xlabel(r"log($M_{\rm subhalo}/M_{\odot}$)", fontsize=14)
plt.ylabel(r"log($M_{\bigstar}/M_{\odot}$)", fontsize=14)
plt.grid(alpha=0.15)

plt.text(0.95, 0.2, "Centrals", color="C3", fontsize=16, transform=plt.gca().transAxes, ha="right")
plt.text(0.95, 0.1, "Satellites", color="C0", fontsize=16, transform=plt.gca().transAxes, ha="right")

plt.savefig(RESULTS_DIR / "figures/smhr.pdf")

plt.clf()

<Figure size 1200x1200 with 0 Axes>

## Plot merger tree

In [49]:
tree = tree_data[100]

plot_merger_tree(
    tree, 
    save_figname=f"tree_{tree.root_subhalo_id}", 
    title=f"Root Subhalo ID: {tree.root_subhalo_id}",
    vmax=11
)

plt.clf();

<Figure size 1800x2400 with 0 Axes>

In [50]:
ttree = trimmed_trees[100]

plot_merger_tree(
    ttree, 
    save_figname=f"trimmed_tree_{ttree.root_subhalo_id}", 
    title=f"Root Subhalo ID: {ttree.root_subhalo_id} (trimmed)",
    vmax=11
)

plt.clf();

<Figure size 1800x2400 with 0 Axes>

In [60]:
tree_stump = tree_stumps[100]

plot_merger_tree(
    tree_stump, 
    save_figname=f"tree_stump_{ttree.root_subhalo_id}", 
    title=f"Root Subhalo ID: {ttree.root_subhalo_id} (stump)",
    vmax=11
)

plt.clf();

<Figure size 1800x2400 with 0 Axes>

## Plot env point cloud

In [24]:
from mpl_toolkits.mplot3d import Axes3D


In [26]:
fig = plt.figure(figsize=(12, 10), dpi=150)
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter3D(
    *env_data.pos.T.numpy(), 
    c=env_data.y[:, 0].numpy().flatten(), 
    s=(env_data.y[:, 0].numpy().flatten() - 8) ** 3, 
    vmin=8.5, 
    vmax=11,
    cmap='magma',
    rasterized=True
)

cb = plt.colorbar(sc, aspect=50, shrink=0.8)
cb.set_label(r'log($M_{\rm subhalo}/M_{\odot}$)', fontsize=12)

ax.set_xlabel('X [Mpc]', fontsize=12)
ax.set_ylabel('Y [Mpc]', fontsize=12)
ax.set_zlabel('Z [Mpc]', fontsize=12)
ax.set_title('TNG Subhalos', fontsize=16)

plt.savefig(RESULTS_DIR / "figures/cosmic-graph.pdf")

plt.clf()

/home/john/miniforge3/envs/pyg/lib/python3.11/site-packages/matplotlib/collections.py:996: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor


<Figure size 1800x1500 with 0 Axes>